In [ ]:
import ee
import geemap
from typing import Dict, List, Tuple, Optional, Union
import os
from IPython.display import display, HTML

class ChilikaLakeBoundary:
    """
    Precise Chilika Lake boundary definition with visualization capabilities.
    
    This class provides an accurate representation of Chilika Lake boundaries
    based on the FAO GAUL (Global Administrative Unit Layers) dataset and
    satellite imagery. Chilika Lake is located in Odisha, India and is
    the largest coastal lagoon in India and second largest brackish water
    lagoon in the world.
    
    Attributes:
        boundary (ee.Geometry): Earth Engine geometry representing the Chilika Lake boundary
        simplified_boundary (ee.Geometry): Simplified version of the boundary for faster processing
    """
    
    def __init__(self, project_id: Optional[str] = None):
        """
        Initialize the Chilika Lake boundary definition.
        
        Args:
            project_id (Optional[str]): Earth Engine project ID for authentication.
                                        If None, uses default credentials.
        
        Raises:
            RuntimeError: If Earth Engine initialization fails.
        """
        # Initialize Earth Engine
        self._initialize_earth_engine(project_id)
        
        # Create the boundary
        self.boundary = self._create_precise_boundary()
        
        # Create a simplified version for faster operations when detail isn't critical
        # Use an explicit error margin to avoid precision issues
        self.simplified_boundary = self.boundary.simplify(maxError=100).buffer(0, 100)
    
    def _initialize_earth_engine(self, project_id: Optional[str] = None) -> None:
        """
        Initialize Earth Engine with appropriate authentication.
        
        This method attempts direct initialization first and falls back to
        authentication if needed, handling project-specific initialization.
        
        Args:
            project_id (Optional[str]): Earth Engine project ID for authentication.
                                       If None, uses default credentials.
        
        Raises:
            RuntimeError: If initialization fails after authentication attempt.
        """
        try:
            if project_id:
                ee.Initialize(project=project_id)
                print(f"✅ Earth Engine initialized successfully with project: {project_id}")
            else:
                ee.Initialize()
                print("✅ Earth Engine initialized successfully with default credentials")
        except Exception as e:
            print(f"🔐 Authentication required: {str(e)}")
            try:
                ee.Authenticate()
                if project_id:
                    ee.Initialize(project=project_id)
                else:
                    ee.Initialize()
                print("✅ Earth Engine initialized after authentication")
            except Exception as auth_error:
                raise RuntimeError(f"Failed to initialize Earth Engine: {str(auth_error)}")
    
    def _create_precise_boundary(self):
        """
        Create a precise boundary for Chilika Lake using FAO GAUL dataset and water mask.
        
        Returns:
            ee.Geometry: The Chilika Lake boundary as an Earth Engine geometry object
        """
        # Load FAO GAUL dataset for administrative boundaries
        self.gaul = ee.FeatureCollection("FAO/GAUL/2015/level2")
        
        # Filter to Odisha, India
        self.odisha_districts = self.gaul.filter(ee.Filter.eq('ADM1_NAME', 'Odisha'))
        
        # Get the specific districts that contain Chilika Lake (Puri, Khordha, and Ganjam)
        chilika_districts = self.odisha_districts.filter(
            ee.Filter.inList('ADM2_NAME', ['Puri', 'Khordha', 'Ganjam'])
        )
        
        # Create a bounding box specifically for Chilika Lake region
        chilika_bbox = ee.Geometry.Rectangle([85.15, 19.45, 85.55, 19.85])
        
        # Use JRC Global Surface Water dataset to identify water bodies in the region
        gsw = ee.Image("JRC/GSW1_3/GlobalSurfaceWater")
        
        # Define permanent water (present in >80% of observations)
        water_mask = gsw.select('occurrence').gt(80)
        
        # Mask to get just the water areas
        chilika_water = water_mask.selfMask()
        
        # Clip to our area of interest
        chilika_water_clipped = chilika_water.clip(chilika_bbox)
        
        # Convert water pixels to vectors using proper parameter format
        vectors = chilika_water_clipped.reduceToVectors(
            reducer=ee.Reducer.countEvery(),
            geometry=chilika_bbox,
            scale=30,
            maxPixels=1e9
        )
        
        try:
            # Get the largest water body (which should be Chilika Lake)
            chilika_feature = vectors.sort('count', False).first()
            
            # Some processing to smooth and simplify the boundary
            chilika_geometry = chilika_feature.geometry()
            
            print("✅ Successfully extracted Chilika Lake boundary from FAO GAUL and water data")
            
            return chilika_geometry
            
        except Exception as e:
            print(f"⚠️ Error extracting boundary from water data: {str(e)}")
            print("⚠️ Falling back to district-based boundary")
            
            # Fallback: Use the combined geometry of the districts that contain Chilika Lake
            # and clip to our bounding box of interest
            district_boundary = chilika_districts.geometry().intersection(chilika_bbox)
            return district_boundary
    
    def display_boundary(self, basemap: str = 'HYBRID') -> geemap.Map:
        """
        Creates and displays an interactive map of the Chilika Lake boundary.
        
        This method renders the boundary on an interactive web map with customizable
        basemap options and visual styling to enhance visibility.
        
        Args:
            basemap (str): Basemap type to use as background. Options include:
                          'HYBRID', 'SATELLITE', 'ROADMAP', 'TERRAIN'.
                          Default is 'HYBRID'.
        
        Returns:
            geemap.Map: Interactive map object with the Chilika Lake boundary displayed
        """
        # Create a map centered on Chilika Lake
        map_cl = geemap.Map()
        map_cl.centerObject(self.boundary, 10)  # Zoom level 10 for a lake-scale view
        
        # Add the Chilika Lake boundary with blue color and styling
        map_cl.addLayer(
            self.boundary, 
            {'color': '0000FF', 'fillColor': '0000FF33', 'width': 2}, 
            'Chilika Lake'
        )
        
        # Add the simplified boundary for comparison (optional)
        map_cl.addLayer(
            self.simplified_boundary,
            {'color': 'FF0000', 'width': 1},
            'Simplified Boundary (for processing)',
            False  # Initially hidden
        )
        
        # Add FAO GAUL administrative boundaries for context
        # Use the already loaded districts from initialization
        map_cl.addLayer(
            self.odisha_districts, 
            {'color': '000000', 'fillColor': '00000000'}, 
            'Odisha Districts',
            False  # Initially hidden
        )
        
        # Add basemap layers for context
        map_cl.add_basemap(basemap)
        
        # Add measurement tools
        map_cl.add_inspector()
        
        # Add a legend
        legend_dict = {
            'Chilika Lake': '0000FF',
            'District Boundaries': '000000'
        }
        map_cl.add_legend(title="Legend", legend_dict=legend_dict)
        
        # Display the map
        display(map_cl)
        
        return map_cl
    
    def get_area(self, units: str = 'km²') -> float:
        """
        Calculate the total area of Chilika Lake.
        
        Args:
            units (str): Units for area calculation. Options: 'km²', 'm²', 'ha'.
                        Default is 'km²'.
                        
        Returns:
            float: Area of Chilika Lake in specified units
            
        Raises:
            ValueError: If an invalid unit is specified
        """
        try:
            # Apply an error margin for the area calculation to prevent precision errors
            # This addresses the "Geometry.area: Unable to perform this geometry operation" error
            geometry_with_margin = self.boundary.buffer(1)  # 1m buffer
            
            # Calculate area in square meters with error margin
            area_m2 = geometry_with_margin.area(1).getInfo()  # Pass error margin of 1
            
            # Convert to requested units
            if units == 'm²':
                return area_m2
            elif units == 'km²':
                return area_m2 / 1e6
            elif units == 'ha':
                return area_m2 / 1e4
            else:
                raise ValueError(f"Invalid unit: {units}. Valid options are: 'm²', 'km²', 'ha'")
        except Exception as e:
            print(f"⚠️ Error calculating area: {str(e)}")
            print("⚠️ Returning approximate area based on simplified geometry")
            
            # Fallback to simplified geometry with explicit error margin
            try:
                # Use a more aggressive simplification with error margin
                simplified = self.boundary.simplify(maxError=100)
                area_m2 = simplified.area(100).getInfo()  # Large error margin
                
                # Convert to requested units
                if units == 'm²':
                    return area_m2
                elif units == 'km²':
                    return area_m2 / 1e6
                elif units == 'ha':
                    return area_m2 / 1e4
                else:
                    raise ValueError(f"Invalid unit: {units}. Valid options are: 'm²', 'km²', 'ha'")
            except Exception as e2:
                print(f"⚠️ Could not calculate area: {str(e2)}")
                # Return an approximate area based on known size of Chilika Lake
                return 1165  # ~1165 km² is the documented area of Chilika Lake
    
    def export_boundary(self, format_type: str = 'geojson', filename: str = 'chilika_lake_boundary') -> str:
        """
        Export the Chilika Lake boundary to a file.
        
        Args:
            format_type (str): Format to export. Options: 'geojson', 'kml', 'shp'.
                             Default is 'geojson'.
            filename (str): Base filename without extension. 
                           Default is 'chilika_lake_boundary'.
                           
        Returns:
            str: Path to the exported file
            
        Raises:
            ValueError: If an invalid format type is specified
            IOError: If the export operation fails
        """
        try:
            # Create export directory if it doesn't exist
            os.makedirs('exports', exist_ok=True)
            
            # Full path with appropriate extension
            if format_type == 'geojson':
                ext = 'geojson'
            elif format_type == 'kml':
                ext = 'kml'
            elif format_type == 'shp':
                ext = 'shp'
            else:
                raise ValueError(f"Invalid format type: {format_type}. Valid options are: 'geojson', 'kml', 'shp'")
                
            filepath = f"exports/{filename}.{ext}"
            
            # Export the boundary
            if format_type == 'geojson':
                geom_json = self.boundary.getInfo()
                import json
                with open(filepath, 'w') as f:
                    json.dump(geom_json, f)
            elif format_type == 'kml':
                # For KML export, we could use Earth Engine's Export features
                # This is a simplified approach for demonstration
                raise NotImplementedError("KML export not implemented yet")
            elif format_type == 'shp':
                # Shapefile export would require additional libraries
                raise NotImplementedError("Shapefile export not implemented yet")
            
            print(f"✅ Exported boundary to {filepath}")
            return filepath
            
        except Exception as e:
            raise IOError(f"Failed to export boundary: {str(e)}")

    def get_coordinates(self) -> List[List[float]]:
        """
        Get the raw coordinates defining the Chilika Lake boundary.
        
        This method returns the exact coordinate points used to define the boundary,
        which can be useful for manual verification or for exporting to other 
        geospatial processing systems.
        
        Returns:
            List[List[float]]: List of [longitude, latitude] coordinate pairs
        """
        try:
            # Extract coordinates from the Earth Engine geometry
            geom_json = self.boundary.getInfo()
            
            # Navigate the GeoJSON structure to extract coordinates
            # Note: This assumes a simple polygon; complex geometries would need more processing
            if 'coordinates' in geom_json:
                return geom_json['coordinates'][0]  # First polygon's coordinates
            else:
                # Fallback to regenerating coordinates
                return self._create_precise_boundary().getInfo()['coordinates'][0]
        except Exception as e:
            print(f"Warning: Could not extract coordinates: {str(e)}")
            return []


# Example usage - simplified for immediate application in a Jupyter Notebook cell
def create_chilika_lake_map(project_id=None):
    """
    Create and display the Chilika Lake boundary map.
    
    This function serves as a simplified entry point for direct usage in a
    Jupyter Notebook cell, handling initialization and visualization in one step.
    
    Args:
        project_id (Optional[str]): Earth Engine project ID for authentication.
                                    Default is None (uses default credentials).
                                    
    Returns:
        Tuple[ChilikaLakeBoundary, geemap.Map]: The boundary object and map display
    """
    try:
        # Create boundary object
        cl_boundary = ChilikaLakeBoundary(project_id)
        
        # Display the boundary
        map_display = cl_boundary.display_boundary(basemap='HYBRID')
        
        # Print area information - wrapped in try/except to handle potential errors
        try:
            area_km2 = cl_boundary.get_area('km²')
            print(f"Chilika Lake Area: {area_km2:,.2f} km²")
        except Exception as e:
            print(f"Note: Could not calculate exact area. Using approximate value.")
            print(f"Chilika Lake Area: ~1,165.00 km² (approximate)")
        
        return cl_boundary, map_display
    
    except Exception as e:
        print(f"Error creating Chilika Lake map: {str(e)}")
        print("Consider checking your Earth Engine authentication and trying again.")


# Run the function to create and display the Chilika Lake map
cl_boundary, map_display = create_chilika_lake_map(project_id='ee-dhanikondav')

✅ Earth Engine initialized successfully with project: ee-dhanikondav
✅ Successfully extracted Chilika Lake boundary from FAO GAUL and water data


c:\Users\dhani\AppData\Local\Programs\Python\Python311\Lib\site-packages\ee\deprecation.py:207: DeprecationWarning: 

Attention required for JRC/GSW1_3/GlobalSurfaceWater! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_3_GlobalSurfaceWater

  warnings.warn(warning, category=DeprecationWarning)


Map(center=[19.717498382922464, 85.34052808892599], controls=(WidgetControl(options=['position', 'transparent_…

Chilika Lake Area: 620.30 km²


EEException: Element.geometry: Parameter 'feature' is required and may not be null.

EEException: Element.geometry: Parameter 'feature' is required and may not be null.

EEException: Element.geometry: Parameter 'feature' is required and may not be null.

EEException: Element.geometry: Parameter 'feature' is required and may not be null.

EEException: Element.geometry: Parameter 'feature' is required and may not be null.

In [6]:
# Cell 2: Add specialized functions for carbon sink analysis

class ChilikaLakeImagery:
    """
    Satellite imagery processing for Chilika Lake boundary with GEE.
    
    This class provides specialized functions for analyzing the Chilika Lake
    using various satellite imagery sources including Sentinel-2, Landsat-8,
    and MODIS. It supports cloud masking, composite generation, water quality
    analysis, and time series extraction.
    """
    
    def __init__(self, boundary_obj, start_date='2015-01-01', end_date='2025-01-01', scale=30):
        """
        Initialize imagery analysis with boundary, date range, and scale.
        
        Args:
            boundary_obj: ChilikaLakeBoundary object containing the lake geometry
            start_date (str): Start date for analysis (YYYY-MM-DD)
            end_date (str): End date for analysis (YYYY-MM-DD)
            scale (int): Analysis scale in meters (default 30m for detailed lake analysis)
        """
        self.boundary = boundary_obj.boundary
        self.start_date = start_date
        self.end_date = end_date
        self.scale = scale
        
        # Define collections appropriate for lake/water body analysis
        self.collections = {
            'sentinel2': ee.ImageCollection('COPERNICUS/S2_SR'),  # 10m resolution, good for lakes
            'landsat8': ee.ImageCollection('LANDSAT/LC08/C02/T1_L2'),  # 30m resolution
            'modis': ee.ImageCollection('MODIS/006/MOD09GA'),  # Lower resolution but good temporal coverage
            'sentinel1': ee.ImageCollection('COPERNICUS/S1_GRD')  # Radar imagery, useful for water mapping
        }
        
        # Water-specific indices
        self.water_indices = {
            'ndwi': 'Normalized Difference Water Index',
            'mndwi': 'Modified Normalized Difference Water Index',
            'awei': 'Automated Water Extraction Index',
            'turbidity': 'Water Turbidity Index'
        }
        
    def _apply_cloud_masking(self, collection_name, collection):
        """
        Apply cloud masking based on collection type, optimized for lake analysis.
        
        Args:
            collection_name (str): Name of the collection
            collection (ee.ImageCollection): The image collection to mask
            
        Returns:
            ee.ImageCollection: Cloud-masked collection
        """
        if collection_name == 'sentinel2':
            # Sentinel-2 cloud masking - using SCL band for more precise cloud detection
            def mask_s2_clouds(image):
                # Get the SCL band and create a mask for clear land/water pixels
                scl = image.select('SCL')
                clear_mask = scl.eq(4).Or(scl.eq(5)).Or(scl.eq(6)).Or(scl.eq(7))
                
                # Use QA60 as a backup for earlier imagery without SCL
                qa60 = image.select('QA60')
                cloud_mask = qa60.bitwiseAnd(1 << 10).eq(0)
                
                # Combine masks for more robust cloud filtering
                final_mask = clear_mask.Or(cloud_mask)
                
                return image.updateMask(final_mask)
            
            return collection.map(mask_s2_clouds)
            
        elif collection_name == 'landsat8':
            # Landsat 8 cloud masking - using QA_PIXEL for detailed cloud/shadow masking
            def mask_l8_clouds(image):
                qa = image.select('QA_PIXEL')
                
                # Bits 3 and 4 are cloud and cloud shadow respectively
                cloud_mask = qa.bitwiseAnd(1 << 3).eq(0)
                shadow_mask = qa.bitwiseAnd(1 << 4).eq(0)
                
                # Combine masks
                mask = cloud_mask.And(shadow_mask)
                
                return image.updateMask(mask)
            
            return collection.map(mask_l8_clouds)
            
        elif collection_name == 'modis':
            # MODIS cloud masking
            def mask_modis_clouds(image):
                qa = image.select('state_1km')
                cloud_mask = qa.bitwiseAnd(1).eq(0)
                return image.updateMask(cloud_mask)
            
            return collection.map(mask_modis_clouds)
            
        elif collection_name == 'sentinel1':
            # Sentinel-1 (radar) doesn't need cloud masking, but we can filter for quality
            def filter_s1_quality(image):
                # Remove low-quality border pixels that commonly occur in Sentinel-1
                edge_mask = image.select(0).mask().updateMask(image.select(0).gt(-30))
                return image.updateMask(edge_mask)
            
            return collection.map(filter_s1_quality)
            
        return collection
    
    def _calculate_water_indices(self, image, collection_name):
        """
        Calculate water-specific indices for water quality analysis.
        
        Args:
            image (ee.Image): Input satellite image
            collection_name (str): Source collection name
            
        Returns:
            ee.Image: Image with added water index bands
        """
        if collection_name == 'sentinel2':
            # NDWI (Normalized Difference Water Index) using Green and NIR
            ndwi = image.normalizedDifference(['B3', 'B8']).rename('ndwi')
            
            # MNDWI (Modified NDWI) using Green and SWIR
            mndwi = image.normalizedDifference(['B3', 'B11']).rename('mndwi')
            
            # Turbidity index (approximation using Red band)
            turbidity = image.select('B4').divide(10000).rename('turbidity')
            
            # Add all indices to the image
            return image.addBands([ndwi, mndwi, turbidity])
            
        elif collection_name == 'landsat8':
            # NDWI for Landsat 8
            ndwi = image.normalizedDifference(['SR_B3', 'SR_B5']).rename('ndwi')
            
            # MNDWI for Landsat 8
            mndwi = image.normalizedDifference(['SR_B3', 'SR_B6']).rename('mndwi')
            
            # Turbidity approximation
            turbidity = image.select('SR_B4').multiply(0.0000275).add(-0.2).rename('turbidity')
            
            return image.addBands([ndwi, mndwi, turbidity])
            
        elif collection_name == 'modis':
            # Simplified NDWI for MODIS
            ndwi = image.normalizedDifference(['sur_refl_b04', 'sur_refl_b02']).rename('ndwi')
            
            # Turbidity approximation
            turbidity = image.select('sur_refl_b01').divide(10000).rename('turbidity')
            
            return image.addBands([ndwi, turbidity])
            
        elif collection_name == 'sentinel1':
            # For Sentinel-1, use VV/VH ratio as water indicator
            water_indicator = image.select('VV').divide(image.select('VH')).rename('water_indicator')
            
            return image.addBands(water_indicator)
            
        return image
    
    def _create_seasonal_composites(self, collection_name):
        """
        Create seasonal composites for lake analysis, accounting for monsoon seasons.
        
        Args:
            collection_name (str): Name of the collection to process
            
        Returns:
            ee.ImageCollection: Collection of seasonal composites
        """
        # Filter collection by date and boundary
        filtered = self.collections[collection_name] \
            .filterDate(self.start_date, self.end_date) \
            .filterBounds(self.boundary)
        
        # Apply cloud masking
        masked = self._apply_cloud_masking(collection_name, filtered)
        
        # Define seasons based on Indian monsoon patterns
        # Chilika Lake's hydrology is strongly influenced by monsoon
        seasons = [
            {'name': 'winter', 'start_month': 12, 'end_month': 2},  # Winter (Dec-Feb)
            {'name': 'summer', 'start_month': 3, 'end_month': 5},   # Summer (Mar-May)
            {'name': 'monsoon', 'start_month': 6, 'end_month': 9},  # Monsoon (Jun-Sep)
            {'name': 'post-monsoon', 'start_month': 10, 'end_month': 11}  # Post-monsoon (Oct-Nov)
        ]
        
        # Generate list of years to process
        start_year = ee.Date(self.start_date).get('year').getInfo()
        end_year = ee.Date(self.end_date).get('year').getInfo()
        years = list(range(start_year, end_year + 1))
        
        # Create composites for each season and year
        composites_list = []
        
        for year in years:
            for season in seasons:
                # Handle winter season that spans two years
                if season['name'] == 'winter':
                    season_start = f"{year-1}-{season['start_month']:02d}-01"
                    season_end = f"{year}-{season['end_month']:02d}-28"
                else:
                    season_start = f"{year}-{season['start_month']:02d}-01"
                    season_end = f"{year}-{season['end_month']:02d}-30"
                
                # Filter images for this season
                season_images = masked.filterDate(season_start, season_end)
                
                # Skip if no images available
                if season_images.size().getInfo() == 0:
                    continue
                
                # Create median composite
                composite = season_images.median()
                
                # Calculate water indices
                composite = self._calculate_water_indices(composite, collection_name)
                
                # Add metadata
                labeled_composite = composite.set({
                    'system:time_start': ee.Date(season_start).millis(),
                    'season': season['name'],
                    'year': year,
                    'period': f"{year}_{season['name']}"
                })
                
                composites_list.append(labeled_composite)
        
        # Create image collection from list
        composites = ee.ImageCollection.fromImages(composites_list)
        
        return composites
    
    def generate_all_composites(self):
        """
        Generate composites for all collections.
        
        Returns:
            dict: Dictionary of composites for each collection
        """
        composites = {}
        for collection_name in self.collections.keys():
            composites[collection_name] = self._create_seasonal_composites(collection_name)
        
        return composites
    
    def display_water_extent(self, collection_name='sentinel2', year=2020, season='monsoon'):
        """
        Display water extent for a specified season and year.
        
        Args:
            collection_name (str): Name of collection to use
            year (int): Year to display
            season (str): Season to display ('winter', 'summer', 'monsoon', 'post-monsoon')
            
        Returns:
            geemap.Map: Map object with water extent displayed
        """
        # Generate composites if not done already
        composites = self._create_seasonal_composites(collection_name)
        
        # Filter to specific season/year
        period = f"{year}_{season}"
        composite = composites.filter(ee.Filter.eq('period', period)).first()
        
        # Create map
        map_water = geemap.Map()
        map_water.centerObject(self.boundary, 11)  # Higher zoom level for lake detail
        
        # Add the lake boundary
        map_water.addLayer(
            self.boundary, 
            {'color': 'red', 'fillColor': '00000000', 'width': 2}, 
            'Chilika Lake Boundary'
        )
        
        # Add standard RGB visualization
        if collection_name == 'sentinel2':
            map_water.addLayer(
                composite, 
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000, 'gamma': 1.4},
                f'Sentinel-2 RGB {year} {season}'
            )
            
            # Add NDWI for water extent
            map_water.addLayer(
                composite.select('ndwi'), 
                {'min': -0.5, 'max': 0.5, 'palette': ['tan', 'white', 'cyan', 'blue']},
                f'Water Extent (NDWI) {year} {season}'
            )
            
            # Add MNDWI for clearer water boundaries
            map_water.addLayer(
                composite.select('mndwi'), 
                {'min': -0.5, 'max': 0.5, 'palette': ['brown', 'white', 'cyan', 'darkblue']},
                f'Water Extent (MNDWI) {year} {season}',
                False  # Initially hidden
            )
            
            # Add turbidity visualization
            map_water.addLayer(
                composite.select('turbidity'), 
                {'min': 0, 'max': 0.3, 'palette': ['blue', 'cyan', 'yellow', 'red']},
                f'Water Turbidity {year} {season}',
                False  # Initially hidden
            )
            
        elif collection_name == 'landsat8':
            map_water.addLayer(
                composite, 
                {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 30000, 'gamma': 1.4},
                f'Landsat 8 RGB {year} {season}'
            )
            
            # Add NDWI for water extent
            map_water.addLayer(
                composite.select('ndwi'), 
                {'min': -0.5, 'max': 0.5, 'palette': ['tan', 'white', 'cyan', 'blue']},
                f'Water Extent (NDWI) {year} {season}'
            )
        
        elif collection_name == 'sentinel1':
            # For Sentinel-1, use VV polarization for water detection
            map_water.addLayer(
                composite, 
                {'bands': ['VV'], 'min': -25, 'max': 0, 'gamma': 1.2},
                f'Sentinel-1 VV {year} {season}'
            )
            
            # Add VV/VH ratio visualization
            map_water.addLayer(
                composite.select('water_indicator'), 
                {'min': 0.5, 'max': 2, 'palette': ['blue', 'cyan', 'yellow', 'red']},
                f'Water Indicator {year} {season}'
            )
        
        # Add tools and legend
        map_water.add_inspector()
        map_water.add_layer_manager()
        
        # Add a legend for water indices
        legend_dict = {
            'Deep Water': 'blue',
            'Shallow Water': 'cyan',
            'Wet Soil': 'lightblue',
            'Vegetation': 'green',
            'Dry Land': 'brown'
        }
        map_water.add_legend(title="Water Extent", legend_dict=legend_dict)
        
        return map_water
    
    def analyze_water_quality(self, year_range=None, collection_name='sentinel2'):
        """
        Analyze water quality of Chilika Lake over time.
        
        Args:
            year_range (list): List of [start_year, end_year]
            collection_name (str): Collection to use for analysis
            
        Returns:
            geemap.Map: Map with water quality visualization
        """
        if year_range is None:
            year_range = [2015, 2025]
        
        # Filter collection to date range and boundary
        filtered = self.collections[collection_name] \
            .filterDate(f"{year_range[0]}-01-01", f"{year_range[1]}-01-01") \
            .filterBounds(self.boundary)
        
        # Apply cloud masking
        masked = self._apply_cloud_masking(collection_name, filtered)
        
        # Create a median composite for the period
        if collection_name == 'sentinel2':
            # Calculate water quality indicators for Sentinel-2
            composite = masked.median()
            
            # Calculate indices
            ndwi = composite.normalizedDifference(['B3', 'B8']).rename('ndwi')
            mndwi = composite.normalizedDifference(['B3', 'B11']).rename('mndwi')
            
            # Turbidity using red band (B4)
            turbidity = composite.select('B4').divide(10000).rename('turbidity')
            
            # Chlorophyll approximation using B5/B4 ratio (red edge/red)
            chlorophyll = composite.select('B5').divide(composite.select('B4')).rename('chlorophyll')
            
            # Combine into a water quality index (simplified)
            # Higher NDWI + lower turbidity + normalized chlorophyll = better water quality
            water_quality = ndwi.add(1) \
                           .subtract(turbidity.multiply(3)) \
                           .add(chlorophyll.multiply(0.5)) \
                           .rename('water_quality')
            
            # Clip to water areas only using MNDWI > 0 as water mask
            water_mask = mndwi.gt(0)
            water_quality = water_quality.updateMask(water_mask)
            
        elif collection_name == 'landsat8':
            # Similar approach for Landsat 8
            composite = masked.median()
            
            # Calculate simplified water quality
            ndwi = composite.normalizedDifference(['SR_B3', 'SR_B5']).rename('ndwi')
            turbidity = composite.select('SR_B4').multiply(0.0000275).add(-0.2).rename('turbidity')
            
            water_quality = ndwi.add(1).subtract(turbidity.multiply(2)).rename('water_quality')
            water_mask = ndwi.gt(0)
            water_quality = water_quality.updateMask(water_mask)
        
        else:
            # For other sensors, use simplified approach
            composite = masked.median()
            water_quality = composite.select(0).multiply(0).add(1).rename('water_quality')
            print(f"Detailed water quality analysis not available for {collection_name}")
        
        # Create visualization
        map_quality = geemap.Map()
        map_quality.centerObject(self.boundary, 11)
        
        # Add the lake boundary
        map_quality.addLayer(
            self.boundary, 
            {'color': 'black', 'fillColor': '00000000', 'width': 2}, 
            'Chilika Lake Boundary'
        )
        
        # Add water quality visualization
        map_quality.addLayer(
            water_quality, 
            {
                'min': 0, 
                'max': 2.5, 
                'palette': ['red', 'orange', 'yellow', 'lightgreen', 'green']
            },
            f'Water Quality Index {year_range[0]}-{year_range[1]}'
        )
        
        # Add legend
        legend_dict = {
            'Poor Quality': 'red',
            'Fair Quality': 'orange',
            'Moderate Quality': 'yellow',
            'Good Quality': 'lightgreen',
            'Excellent Quality': 'green'
        }
        map_quality.add_legend(title="Water Quality", legend_dict=legend_dict)
        
        return map_quality
    
    def analyze_seasonal_changes(self, year=2020, collection_name='sentinel2'):
        """
        Analyze seasonal changes in Chilika Lake within a year.
        
        Args:
            year (int): Year to analyze
            collection_name (str): Collection to use
            
        Returns:
            geemap.Map: Map with seasonal comparison
        """
        # Generate seasonal composites
        composites = self._create_seasonal_composites(collection_name)
        
        # Filter to the specified year
        year_composites = composites.filter(ee.Filter.eq('year', year))
        
        # Create map
        map_seasons = geemap.Map()
        map_seasons.centerObject(self.boundary, 11)
        
        # Add the lake boundary
        map_seasons.addLayer(
            self.boundary, 
            {'color': 'black', 'fillColor': '00000000', 'width': 2}, 
            'Chilika Lake Boundary'
        )
        
        # Add each season
        seasons = ['winter', 'summer', 'monsoon', 'post-monsoon']
        
        for season in seasons:
            # Try to get the composite for this season
            season_composite = year_composites.filter(ee.Filter.eq('season', season)).first()
            
            # Skip if no data
            if season_composite is None:
                print(f"No data available for {season} {year}")
                continue
                
            if collection_name == 'sentinel2':
                # Add RGB visualization
                map_seasons.addLayer(
                    season_composite, 
                    {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000, 'gamma': 1.4},
                    f'RGB {season} {year}',
                    False  # Initially hidden
                )
                
                # Add NDWI for water extent (showing seasonal variation)
                map_seasons.addLayer(
                    season_composite.select('ndwi'), 
                    {'min': -0.5, 'max': 0.5, 'palette': ['tan', 'white', 'cyan', 'blue']},
                    f'Water Extent {season} {year}'
                )
                
            elif collection_name == 'landsat8':
                # Similar approach for Landsat 8
                map_seasons.addLayer(
                    season_composite, 
                    {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 30000, 'gamma': 1.4},
                    f'RGB {season} {year}',
                    False  # Initially hidden
                )
                
                map_seasons.addLayer(
                    season_composite.select('ndwi'), 
                    {'min': -0.5, 'max': 0.5, 'palette': ['tan', 'white', 'cyan', 'blue']},
                    f'Water Extent {season} {year}'
                )
        
        # Add legend
        legend_dict = {
            'Deep Water': 'blue',
            'Shallow Water': 'cyan',
            'Wet Soil': 'lightblue',
            'Dry Land': 'tan'
        }
        map_seasons.add_legend(title="Seasonal Water Extent", legend_dict=legend_dict)
        
        return map_seasons
    
    def export_water_quality_data(self, format_type='csv', filename='chilika_lake_water_quality'):
        """
        Export water quality time series data.
        
        Args:
            format_type (str): Export format ('csv' or 'json')
            filename (str): Base filename for export
            
        Returns:
            ee.batch.Task: Export task
        """
        # Use Sentinel-2 for best water quality analysis
        composites = self._create_seasonal_composites('sentinel2')
        
        # Extract values for all periods
        def extract_quality_values(image):
            # Get mean values for each index over the lake area
            stats = image.reduceRegion({
                'reducer': ee.Reducer.mean(),
                'geometry': self.boundary,
                'scale': self.scale,
                'maxPixels': 1e9
            })
            
            # Return a feature with properties
            return ee.Feature(None, {
                'period': image.get('period'),
                'season': image.get('season'),
                'year': image.get('year'),
                'ndwi': stats.get('ndwi'),
                'mndwi': stats.get('mndwi'),
                'turbidity': stats.get('turbidity')
            })
        
        # Create feature collection with quality metrics
        quality_series = composites.map(extract_quality_values)
        
        # Export to Google Drive
        task = ee.batch.Export.table.toDrive({
            'collection': quality_series,
            'description': filename,
            'fileFormat': format_type
        })
        
        task.start()
        print(f"Export task started: {filename}.{format_type}")
        
        return task


def calculate_carbon_indices(image, collection_name):
    """
    Calculate specialized indices for carbon sink analysis.
    This focuses on vegetation, algae, and sediment metrics that indicate carbon sequestration.
    
    Args:
        image (ee.Image): Input satellite image
        collection_name (str): Source collection name
        
    Returns:
        ee.Image: Image with added carbon sink index bands
    """
    if collection_name == 'sentinel2':
        # NDVI for terrestrial vegetation around lake
        ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
        
        # Enhanced Vegetation Index (EVI) - better for dense vegetation
        evi = image.expression(
            '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                'NIR': image.select('B8'),
                'RED': image.select('B4'),
                'BLUE': image.select('B2')
            }).rename('evi')
        
        # Floating Algae Index (FAI) - for aquatic vegetation and algae
        fai = image.expression(
            'NIR - (RED + (SWIR - RED) * ((NIR_wl - RED_wl) / (SWIR_wl - RED_wl)))', {
                'NIR': image.select('B8'),
                'RED': image.select('B4'),
                'SWIR': image.select('B11'),
                'NIR_wl': 842,  # Band 8 wavelength
                'RED_wl': 665,  # Band 4 wavelength
                'SWIR_wl': 1610  # Band 11 wavelength
            }).rename('fai')
        
        # Normalized Difference Chlorophyll Index (NDCI)
        ndci = image.normalizedDifference(['B5', 'B4']).rename('ndci')
        
        # Carbon sink potential index (CSPI) - combines vegetation and algae metrics
        cspi = ndvi.add(evi).add(fai.multiply(2)).add(ndci).divide(5).rename('cspi')
        
        # Add all indices to the image
        return image.addBands([ndvi, evi, fai, ndci, cspi])
        
    elif collection_name == 'landsat8':
        # Similar indices for Landsat 8
        ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('ndvi')
        
        evi = image.expression(
            '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                'NIR': image.select('SR_B5'),
                'RED': image.select('SR_B4'),
                'BLUE': image.select('SR_B2')
            }).rename('evi')
        
        # Simplified FAI for Landsat
        fai = image.normalizedDifference(['SR_B5', 'SR_B6']).rename('fai')
        
        # NDCI equivalent for Landsat
        ndci = image.normalizedDifference(['SR_B3', 'SR_B4']).rename('ndci')
        
        # Carbon sink potential index
        cspi = ndvi.add(evi).add(fai).add(ndci).divide(4).rename('cspi')
        
        return image.addBands([ndvi, evi, fai, ndci, cspi])
    
    return image

class ChilikaCarbonSinkAnalysis(ChilikaLakeImagery):
    """
    Extends ChilikaLakeImagery with specialized methods for analyzing
    carbon sink potential of Chilika Lake.
    """
    
    def __init__(self, boundary_obj, start_date='2015-01-01', end_date='2025-01-01', scale=30):
        """Initialize with parent class constructor"""
        super().__init__(boundary_obj, start_date, end_date, scale)
        
        # Update the Sentinel-2 collection to use the latest version with consistent band names
        self.collections['sentinel2'] = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    
    def _calculate_water_indices(self, image, collection_name):
        """Override parent method to add carbon sink indices"""
        # First get standard water indices from parent method
        enhanced_image = super()._calculate_water_indices(image, collection_name)
        
        # Then add our carbon sink specific indices
        return calculate_carbon_indices(enhanced_image, collection_name)
    
    def detect_carbon_sink_zones(self, year=2020, season='monsoon', collection_name='sentinel2'):
        """
        Detect zones of high carbon sequestration potential in Chilika Lake.
        
        Args:
            year (int): Year to analyze
            season (str): Season to analyze
            collection_name (str): Collection to use
            
        Returns:
            geemap.Map: Map showing carbon sink zones
        """
        # Generate composites
        composites = self._create_seasonal_composites(collection_name)
        
        # Filter to specific season/year
        period = f"{year}_{season}"
        composite = composites.filter(ee.Filter.eq('period', period)).first()
        
        # Create map
        map_carbon = geemap.Map()
        map_carbon.centerObject(self.boundary, 11)
        
        # Add the lake boundary
        map_carbon.addLayer(
            self.boundary, 
            {'color': 'black', 'fillColor': '00000000', 'width': 2}, 
            'Chilika Lake Boundary'
        )
        
        # Add standard RGB visualization
        if collection_name == 'sentinel2':
            map_carbon.addLayer(
                composite, 
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000, 'gamma': 1.4},
                f'Sentinel-2 RGB {year} {season}'
            )
            
            # Add NDVI layer
            map_carbon.addLayer(
                composite.select('ndvi'), 
                {'min': -0.2, 'max': 0.8, 'palette': ['blue', 'white', 'yellow', 'green']},
                f'Vegetation Index (NDVI) {year} {season}'
            )
            
            # Add Floating Algae Index layer
            map_carbon.addLayer(
                composite.select('fai'), 
                {'min': -0.05, 'max': 0.1, 'palette': ['darkblue', 'blue', 'cyan', 'yellow', 'red']},
                f'Floating Algae Index {year} {season}'
            )
            
            # Add Carbon Sink Potential Index
            map_carbon.addLayer(
                composite.select('cspi'), 
                {'min': 0, 'max': 0.6, 'palette': ['blue', 'cyan', 'yellow', 'green', 'darkgreen']},
                f'Carbon Sink Potential {year} {season}'
            )
        
        elif collection_name == 'landsat8':
            # Similar visualizations for Landsat 8
            map_carbon.addLayer(
                composite, 
                {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 30000, 'gamma': 1.4},
                f'Landsat 8 RGB {year} {season}'
            )
            
            map_carbon.addLayer(
                composite.select('ndvi'), 
                {'min': -0.2, 'max': 0.8, 'palette': ['blue', 'white', 'yellow', 'green']},
                f'Vegetation Index (NDVI) {year} {season}'
            )
            
            map_carbon.addLayer(
                composite.select('cspi'), 
                {'min': 0, 'max': 0.6, 'palette': ['blue', 'cyan', 'yellow', 'green', 'darkgreen']},
                f'Carbon Sink Potential {year} {season}'
            )
        
        # Add tools
        map_carbon.add_inspector()
        map_carbon.add_layer_manager()
        
        # Add legend - using a proper format with RGB tuples
        legend_items = {
            'High Carbon Sequestration': (0, 100, 0),      # darkgreen
            'Moderate Carbon Seq.': (0, 128, 0),           # green
            'Low Carbon Seq.': (255, 255, 0),              # yellow
            'Very Low Carbon Seq.': (0, 255, 255),         # cyan
            'No Carbon Seq.': (0, 0, 255)                  # blue
        }
        map_carbon.add_legend(title="Carbon Sink Potential", legend_dict=legend_items)
        
        return map_carbon, composite
    
    def analyze_carbon_sequestration_potential(self, year_range=None, collection_name='sentinel2'):
        """
        Analyze the carbon sequestration potential of Chilika Lake over time.
        
        Args:
            year_range (list): List of [start_year, end_year]
            collection_name (str): Collection to use for analysis
            
        Returns:
            tuple: (geemap.Map, ee.Image) - Map and classified carbon sink image
        """
        if year_range is None:
            year_range = [2015, 2023]
        
        # Generate collection filtered to date range and boundary
        filtered = self.collections[collection_name] \
            .filterDate(f"{year_range[0]}-01-01", f"{year_range[1]}-01-01") \
            .filterBounds(self.boundary)
        
        # Apply cloud masking
        masked = self._apply_cloud_masking(collection_name, filtered)
        
        # Create annual composites
        years = range(year_range[0], year_range[1] + 1)
        annual_composites = []
        
        for year in years:
            # Filter to this year
            year_start = f"{year}-01-01"
            year_end = f"{year+1}-01-01"
            year_images = masked.filterDate(year_start, year_end)
            
            # Skip if no images
            if year_images.size().getInfo() == 0:
                print(f"No images available for {year}")
                continue
            
            # Create composite
            composite = year_images.median()
            
            # Calculate indices
            composite = self._calculate_water_indices(composite, collection_name)
            composite = calculate_carbon_indices(composite, collection_name)
            
            # Add metadata
            labeled_composite = composite.set({
                'year': year
            })
            
            annual_composites.append(labeled_composite)
        
        # Create collection from list
        annual_collection = ee.ImageCollection.fromImages(annual_composites)
        
        # Calculate trends in carbon sequestration over time
        if annual_collection.size().getInfo() > 1:
            # Get all CSPI bands
            cspi_collection = annual_collection.select('cspi')
            
            # Calculate trend (slope) of CSPI over time
            # First create an image collection with time band
            def add_time_band(img):
                year = ee.Number(img.get('year'))
                img_time = ee.Image(year.subtract(year_range[0])).float().rename('t')
                return img.addBands(img_time)
            
            time_collection = cspi_collection.map(add_time_band)
            
            # Compute the linear trend using reducer
            trend = time_collection.reduce(ee.Reducer.linearFit())
            
            # Slope is the trend of carbon sequestration (positive = increasing sink)
            slope = trend.select('scale')
            
            # Calculate mean CSPI for the period
            mean_cspi = cspi_collection.mean()
            
            # Create a classified carbon sink map
            # Combine current sink capacity (mean_cspi) with trend (slope)
            # In Earth Engine, we need to use .And() instead of .and()
            carbon_classes = ee.Image(0)
            carbon_classes = carbon_classes.where(mean_cspi.lt(0.1).And(slope.lt(0)), 1)  # Very low and declining
            carbon_classes = carbon_classes.where(mean_cspi.lt(0.2).And(slope.lt(0.001)), 2)  # Low and stable
            carbon_classes = carbon_classes.where(mean_cspi.lt(0.2).And(slope.gt(0.001)), 3)  # Low but improving
            carbon_classes = carbon_classes.where(mean_cspi.gte(0.2).And(mean_cspi.lt(0.4)).And(slope.lt(0)), 4)  # Moderate but declining
            carbon_classes = carbon_classes.where(mean_cspi.gte(0.2).And(mean_cspi.lt(0.4)).And(slope.gte(0)), 5)  # Moderate and stable/improving
            carbon_classes = carbon_classes.where(mean_cspi.gte(0.4).And(slope.lt(-0.002)), 6)  # High but rapidly declining
            carbon_classes = carbon_classes.where(mean_cspi.gte(0.4).And(slope.lt(0)), 7)  # High but slightly declining
            carbon_classes = carbon_classes.where(mean_cspi.gte(0.4).And(slope.gte(0)), 8)  # High and stable/improving
        else:
            # If we don't have enough years for trend analysis, just use the mean value
            mean_cspi = annual_collection.select('cspi').mean()
            
            # Create a simplified classification based only on current values
            carbon_classes = ee.Image(0)
            carbon_classes = carbon_classes.where(mean_cspi.lt(0.1), 1)  # Very low
            carbon_classes = carbon_classes.where(mean_cspi.gte(0.1).And(mean_cspi.lt(0.2)), 2)  # Low
            carbon_classes = carbon_classes.where(mean_cspi.gte(0.2).And(mean_cspi.lt(0.3)), 4)  # Moderate
            carbon_classes = carbon_classes.where(mean_cspi.gte(0.3).And(mean_cspi.lt(0.4)), 5)  # Moderate-High
            carbon_classes = carbon_classes.where(mean_cspi.gte(0.4).And(mean_cspi.lt(0.5)), 7)  # High
            carbon_classes = carbon_classes.where(mean_cspi.gte(0.5), 8)  # Very High
            
            # Set a dummy slope (all 0)
            slope = ee.Image(0).rename('scale')
        
        # Create map
        map_carbon_trend = geemap.Map()
        map_carbon_trend.centerObject(self.boundary, 11)
        
        # Add the lake boundary
        map_carbon_trend.addLayer(
            self.boundary, 
            {'color': 'black', 'fillColor': '00000000', 'width': 2}, 
            'Chilika Lake Boundary'
        )
        
        # Add mean CSPI visualization
        map_carbon_trend.addLayer(
            mean_cspi, 
            {'min': 0, 'max': 0.6, 'palette': ['blue', 'cyan', 'yellow', 'green', 'darkgreen']},
            f'Mean Carbon Sink Potential {year_range[0]}-{year_range[1]}'
        )
        
        # Add slope (trend) visualization if we have it
        if annual_collection.size().getInfo() > 1:
            map_carbon_trend.addLayer(
                slope, 
                {
                    'min': -0.01, 
                    'max': 0.01, 
                    'palette': ['red', 'white', 'green']
                },
                f'Carbon Sink Trend {year_range[0]}-{year_range[1]}'
            )
        
        # Add classified carbon sink map
        map_carbon_trend.addLayer(
            carbon_classes, 
            {
                'min': 1, 
                'max': 8, 
                'palette': ['gray', 'blue', 'cyan', 'lightgreen', 'yellow', 'orange', 'green', 'darkgreen']
            },
            f'Carbon Sink Classification {year_range[0]}-{year_range[1]}'
        )
        
        # Add legend for carbon classification
        legend_items = {
            'Very High & Stable Sink': (0, 100, 0),     # darkgreen
            'High but Declining': (0, 128, 0),          # green
            'Moderate & Improving': (255, 255, 0),      # yellow
            'Moderate but Declining': (255, 165, 0),    # orange
            'Low but Improving': (144, 238, 144),       # lightgreen
            'Low & Stable': (0, 255, 255),              # cyan
            'Very Low': (0, 0, 255),                    # blue
            'Non-functional Sink': (128, 128, 128)      # gray
        }
        map_carbon_trend.add_legend(title="Carbon Sink Classification", legend_items=legend_items)
        
        return map_carbon_trend, carbon_classes
    
    def calculate_carbon_storage(self, collection_name='sentinel2', year=2020, season='monsoon'):
        """
        Calculate estimated carbon storage capacity of different zones in Chilika Lake.
        
        This is an approximation based on vegetation indices, algal presence, and
        wetland characteristics, which correlate with carbon sequestration potential.
        
        Args:
            collection_name (str): Collection to use
            year (int): Year to analyze
            season (str): Season to analyze
            
        Returns:
            dict: Carbon storage estimates and statistics
        """
        # Get composite for the specified period
        composites = self._create_seasonal_composites(collection_name)
        period = f"{year}_{season}"
        composite = composites.filter(ee.Filter.eq('period', period)).first()
        
        # Get CSPI index
        cspi = composite.select('cspi')
        
        # Define carbon storage potentials for different CSPI ranges
        # These are approximate values in tons of CO2 per hectare per year
        # Based on literature for coastal wetlands, mangroves, and lake ecosystems
        carbon_storage = cspi.expression(
            '(cspi < 0.1) * 1 + ' +  # Very low: ~1 tCO2/ha/year
            '(cspi >= 0.1 && cspi < 0.2) * 3 + ' +  # Low: ~3 tCO2/ha/year
            '(cspi >= 0.2 && cspi < 0.3) * 6 + ' +  # Medium-low: ~6 tCO2/ha/year
            '(cspi >= 0.3 && cspi < 0.4) * 10 + ' +  # Medium: ~10 tCO2/ha/year
            '(cspi >= 0.4 && cspi < 0.5) * 15 + ' +  # Medium-high: ~15 tCO2/ha/year
            '(cspi >= 0.5 && cspi < 0.6) * 20 + ' +  # High: ~20 tCO2/ha/year
            '(cspi >= 0.6) * 25',  # Very high: ~25 tCO2/ha/year
            {'cspi': cspi}
        ).rename('carbon_storage')
        
        # Calculate stats for the entire lake
        lake_stats = carbon_storage.reduceRegion(
            reducer=ee.Reducer.mean().combine(
                ee.Reducer.sum(), None, True
            ).combine(
                ee.Reducer.minMax(), None, True
            ),
            geometry=self.boundary,
            scale=self.scale,
            maxPixels=1e9
        ).getInfo()
        
        # Calculate area in hectares
        area_m2 = self.boundary.area(100).getInfo()  # Use error margin of 100m
        area_ha = area_m2 / 10000
        
        # Estimate total carbon sequestration capacity
        # This is annual_rate_per_ha * area_in_ha
        total_annual_seq = lake_stats.get('carbon_storage_sum') * area_ha
        
        # Create a dict with all carbon statistics
        carbon_stats = {
            'area_ha': area_ha,
            'mean_rate_tCO2_per_ha_per_year': lake_stats.get('carbon_storage_mean'),
            'min_rate_tCO2_per_ha_per_year': lake_stats.get('carbon_storage_min'),
            'max_rate_tCO2_per_ha_per_year': lake_stats.get('carbon_storage_max'),
            'total_annual_sequestration_tCO2': total_annual_seq,
            'analysis_year': year,
            'analysis_season': season
        }
        
        return carbon_stats, carbon_storage

In [7]:
# Cell 3: Execute carbon sink analysis cell-by-cell
# Let's use Landsat 8 instead of Sentinel-2 to avoid band compatibility issues

# Initialize the carbon sink analysis with our boundary object
carbon_analysis = ChilikaCarbonSinkAnalysis(cl_boundary, start_date='2015-01-01', end_date='2023-12-31')



In [8]:
# Cell 4: Analyze current carbon sink zones
# This identifies the current carbon sequestration hotspots in the lake
print("Analyzing carbon sink zones for current period...")
carbon_zones_map, carbon_zones_image = carbon_analysis.detect_carbon_sink_zones(
    year=2020,
    season='monsoon',
    collection_name='landsat8'  # Using Landsat 8 instead of Sentinel-2
)
display(carbon_zones_map)


Analyzing carbon sink zones for current period...


Map(center=[19.717498382922464, 85.34052808892599], controls=(WidgetControl(options=['position', 'transparent_…

In [9]:
# Cell 5: Analyze carbon sequestration trends over time
# This identifies how carbon sequestration has changed in different areas
print("Analyzing carbon sequestration trends over time...")
carbon_trend_map, carbon_classes = carbon_analysis.analyze_carbon_sequestration_potential(
    year_range=[2015, 2023],
    collection_name='landsat8'  # Using Landsat 8 instead of Sentinel-2
)
display(carbon_trend_map)

Analyzing carbon sequestration trends over time...
No images available for 2023


Map(center=[19.717498382922464, 85.34052808892599], controls=(WidgetControl(options=['position', 'transparent_…

In [10]:
# Cell 6: Calculate carbon storage capacity
# This provides quantitative estimates of carbon sequestration
print("Calculating carbon storage capacity...")
carbon_stats, carbon_storage = carbon_analysis.calculate_carbon_storage(
    collection_name='landsat8',  # Using Landsat 8 instead of Sentinel-2
    year=2020,
    season='monsoon'
)


Calculating carbon storage capacity...


In [11]:
# Cell 7: Display carbon statistics and storage estimates
print("\nChilika Lake Carbon Sink Statistics:")
print(f"Total Area: {carbon_stats['area_ha']:,.2f} hectares")
print(f"Average Carbon Sequestration Rate: {carbon_stats['mean_rate_tCO2_per_ha_per_year']:.2f} tons CO2/ha/year")
print(f"Minimum Rate: {carbon_stats['min_rate_tCO2_per_ha_per_year']:.2f} tons CO2/ha/year")
print(f"Maximum Rate: {carbon_stats['max_rate_tCO2_per_ha_per_year']:.2f} tons CO2/ha/year")
print(f"Total Annual Carbon Sequestration: {carbon_stats['total_annual_sequestration_tCO2']:,.2f} tons CO2/year")
print(f"Analysis Period: {carbon_stats['analysis_season']} {carbon_stats['analysis_year']}")


Chilika Lake Carbon Sink Statistics:
Total Area: 61,968.77 hectares
Average Carbon Sequestration Rate: 5.69 tons CO2/ha/year
Minimum Rate: 1.00 tons CO2/ha/year
Maximum Rate: 25.00 tons CO2/ha/year
Total Annual Carbon Sequestration: 258,288,207,777.94 tons CO2/year
Analysis Period: monsoon 2020


In [12]:
# Cell 8: Analyze seasonal variations in carbon sink function
# Compare between seasons to understand temporal dynamics
print("\nAnalyzing seasonal variations in carbon sink function...")

# Create dictionaries to store seasonal maps and statistics
seasonal_maps = {}
seasonal_stats = {}

# Loop through each season
for season in ['winter', 'summer', 'monsoon', 'post-monsoon']:
    print(f"Processing {season} season...")
    try:
        # Get carbon sink map for this season
        # Make sure carbon_analysis is properly imported and initialized
        season_map, season_image = carbon_analysis.detect_carbon_sink_zones(
            year=2020,
            season=season,
            collection_name='landsat8'  # Using Landsat 8 instead of Sentinel-2
        )
        seasonal_maps[season] = season_map
        
        # Calculate stats for this season
        season_stats, _ = carbon_analysis.calculate_carbon_storage(
            collection_name='landsat8',  # Using Landsat 8 instead of Sentinel-2
            year=2020,
            season=season
        )
        seasonal_stats[season] = season_stats
        
        # Display the map - check if display function is available in your environment
        print(f"Carbon sink function during {season} season:")
        try:
            display(season_map)  # This might be causing the issue if not in Jupyter
        except NameError:
            print("Map display is only available in Jupyter notebooks")
            
        print(f"Average Rate: {season_stats['mean_rate_tCO2_per_ha_per_year']:.2f} tons CO2/ha/year")
        print(f"Total Sequestration: {season_stats['total_annual_sequestration_tCO2']:,.2f} tons CO2/year")
        print("----------------------------------------------------")
        
    except Exception as e:
        import traceback
        print(f"Could not process {season} season: {str(e)}")
        print(f"Detailed error: {traceback.format_exc()}")


Analyzing seasonal variations in carbon sink function...
Processing winter season...


Carbon sink function during winter season:


Map(center=[19.717498382922464, 85.34052808892599], controls=(WidgetControl(options=['position', 'transparent_…

Average Rate: 5.49 tons CO2/ha/year
Total Sequestration: 237,350,260,149.87 tons CO2/year
----------------------------------------------------
Processing summer season...


Carbon sink function during summer season:


...

Map(center=[19.717498382922464, 85.34052808892599], controls=(WidgetControl(options=['position', 'transparent_…

Average Rate: 5.69 tons CO2/ha/year
Total Sequestration: 258,288,207,777.94 tons CO2/year
----------------------------------------------------
Processing post-monsoon season...


Carbon sink function during post-monsoon season:


Map(center=[19.717498382922464, 85.34052808892599], controls=(WidgetControl(options=['position', 'transparent_…

Average Rate: 5.42 tons CO2/ha/year
Total Sequestration: 246,242,717,351.06 tons CO2/year
----------------------------------------------------


In [13]:
# Cell 9: Create a summary of highest carbon sink areas using optimized approach
print("\nIdentifying highest carbon sink zones in Chilika Lake...")

# Extract high carbon sink areas from the carbon classes image
if 'carbon_classes' in locals():
    # Get the high carbon sink areas (classes 7 and 8)
    high_sink_areas = carbon_classes.gte(7)
    
    try:
        # Method 1: Try to optimize reduceRegion with tileScale to reduce memory usage
        high_sink_stats = high_sink_areas.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=cl_boundary.boundary,
            scale=30,
            maxPixels=1e9,
            tileScale=16  # Increase tileScale to reduce memory per tile
        ).getInfo()
        
        # Calculate percentage of lake that is high carbon sink
        high_sink_pixels = high_sink_stats.get('constant') if 'constant' in high_sink_stats else 0
        high_sink_area_ha = high_sink_pixels * (30 * 30) / 10000  # Convert pixels to hectares
        
    except Exception as e:
        print(f"First method failed with error: {str(e)}")
        print("Trying alternative method...")
        
        # Method 2: Use reduceRegions instead and aggregate after
        try:
            # Convert the image to a binary mask (0 or 1)
            high_sink_mask = high_sink_areas.selfMask()
            
            # Count pixels using pixelArea
            area_image = high_sink_mask.multiply(ee.Image.pixelArea())
            
            # Calculate total area in square meters
            area_stats = area_image.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=cl_boundary.boundary,
                scale=30,
                maxPixels=1e9,
                tileScale=16
            ).getInfo()
            
            # Convert to hectares (1 hectare = 10,000 sq meters)
            high_sink_area_ha = area_stats.get('constant', 0) / 10000
            
        except Exception as e2:
            print(f"Second method also failed: {str(e2)}")
            print("Using simpler estimation method...")
            
            # Method 3: Use sample-based estimation
            try:
                # Sample points across the region
                sample_points = ee.FeatureCollection.randomPoints(
                    region=cl_boundary.boundary,
                    points=500,  # Number of sample points
                    seed=42
                )
                
                # Extract values at each point
                samples = high_sink_areas.sampleRegions(
                    collection=sample_points,
                    scale=30,
                    geometries=True
                )
                
                # Count points with high carbon sink value
                sample_count = samples.aggregate_count('constant').getInfo()
                high_sink_count = samples.filterMetadata('constant', 'equals', 1).size().getInfo()
                
                # Estimate area based on proportion of samples
                total_area_ha = carbon_stats['area_ha']
                high_sink_area_ha = (high_sink_count / sample_count) * total_area_ha
                
                print(f"Using sampling estimation (n={sample_count} points)")
                
            except Exception as e3:
                print(f"All methods failed. Final error: {str(e3)}")
                high_sink_area_ha = 0
                print("Unable to calculate high carbon sink area due to memory limitations.")
    
    # Continue with the analysis if we have a valid area measurement
    if high_sink_area_ha > 0:
        high_sink_percentage = (high_sink_area_ha / carbon_stats['area_ha']) * 100
        
        print(f"High Carbon Sink Area: {high_sink_area_ha:,.2f} hectares")
        print(f"Percentage of Lake: {high_sink_percentage:.2f}%")
        print(f"These areas sequester approximately {high_sink_area_ha * 20:,.2f} tons CO2/year")
        print("(based on average sequestration rate of ~20 tons CO2/ha/year for high sink areas)")
else:
    print("Carbon classes image not available. Unable to identify high sink zones.")


Identifying highest carbon sink zones in Chilika Lake...
High Carbon Sink Area: 15,170.22 hectares
Percentage of Lake: 24.48%
These areas sequester approximately 303,404.40 tons CO2/year
(based on average sequestration rate of ~20 tons CO2/ha/year for high sink areas)


In [14]:
# Cell 10: Export results instead of interactive visualization
print("\nPreparing to export carbon sink analysis results...")

# Create export tasks for each important dataset
if 'carbon_classes' in locals():
    # Define export parameters
    export_params = {
        'scale': 30,
        'region': cl_boundary.boundary,
        'maxPixels': 1e9,
        'fileFormat': 'GeoTIFF'
    }
    
    # 1. Export carbon classification map
    try:
        carbon_task = ee.batch.Export.image.toDrive(
            image=carbon_classes,
            description='chilika_carbon_classes',
            folder='chilika_carbon_analysis',
            **export_params
        )
        carbon_task.start()
        print("Started export of carbon classification map")
        print("Task ID:", carbon_task.id)
        print("Status:", carbon_task.status())
    except Exception as e:
        print(f"Error exporting carbon classes: {str(e)}")
    
    # 2. Export high sink areas as separate file
    if 'high_sink_areas' in locals():
        try:
            high_sink_task = ee.batch.Export.image.toDrive(
                image=high_sink_areas,
                description='chilika_high_carbon_sink',
                folder='chilika_carbon_analysis',
                **export_params
            )
            high_sink_task.start()
            print("\nStarted export of high carbon sink areas")
            print("Task ID:", high_sink_task.id)
            print("Status:", high_sink_task.status())
        except Exception as e:
            print(f"Error exporting high sink areas: {str(e)}")
    
    # 3. Export statistics as CSV file
    if 'carbon_stats' in locals():
        try:
            # Convert stats to feature collection for export
            stats_dict = {
                'total_area_ha': carbon_stats['area_ha'],
                'mean_carbon_rate': carbon_stats['mean_rate_tCO2_per_ha_per_year'],
                'total_sequestration': carbon_stats['total_annual_sequestration_tCO2'],
                'high_sink_area_ha': high_sink_area_ha if 'high_sink_area_ha' in locals() else 0,
                'high_sink_percentage': high_sink_percentage if 'high_sink_percentage' in locals() else 0,
                'high_sink_sequestration': high_sink_area_ha * 20 if 'high_sink_area_ha' in locals() else 0
            }
            
            # Create a feature with the statistics
            stats_feature = ee.Feature(None, stats_dict)
            stats_fc = ee.FeatureCollection([stats_feature])
            
            # Export the feature collection
            stats_task = ee.batch.Export.table.toDrive(
                collection=stats_fc,
                description='chilika_carbon_stats',
                folder='chilika_carbon_analysis',
                fileFormat='CSV'
            )
            stats_task.start()
            print("\nStarted export of carbon statistics")
            print("Task ID:", stats_task.id)
            print("Status:", stats_task.status())
        except Exception as e:
            print(f"Error exporting statistics: {str(e)}")
    
    print("\nExport tasks have been started. You can check the status in the Earth Engine Code Editor Tasks tab.")
    print("Once completed, the files will be available in your Google Drive in the 'chilika_carbon_analysis' folder.")
    
    # Print summary of findings (without visualization)
    print("\n===== CARBON SINK ANALYSIS SUMMARY =====")
    print(f"Total Lake Area: {carbon_stats['area_ha']:,.2f} hectares")
    print(f"Average Carbon Sequestration Rate: {carbon_stats['mean_rate_tCO2_per_ha_per_year']:.2f} tons CO2/ha/year")
    print(f"Total Annual Carbon Sequestration: {carbon_stats['total_annual_sequestration_tCO2']:,.2f} tons CO2/year")
    
    if 'high_sink_area_ha' in locals():
        print(f"High Carbon Sink Area: {high_sink_area_ha:,.2f} hectares ({high_sink_percentage:.2f}% of lake)")
        print(f"High Sink Annual Sequestration: {high_sink_area_ha * 20:,.2f} tons CO2/year")
    
    print("\nRecommendations:")
    print("1. Focus conservation efforts on the high carbon sink areas")
    print("2. Monitor seasonal variations in carbon sequestration")
    print("3. Develop management practices that enhance carbon sink function")
    print("4. Investigate potential for carbon credit generation")
    print("\nAnalysis complete. The exported files can be used for further analysis in GIS software.")
else:
    print("Carbon classes image not available. Unable to export results.")


Preparing to export carbon sink analysis results...
Started export of carbon classification map
Task ID: VZAEB7KLLGCDGBPOF22NSAWH
Status: {'state': 'READY', 'description': 'chilika_carbon_classes', 'priority': 100, 'creation_timestamp_ms': 1741751691677, 'update_timestamp_ms': 1741751691677, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'VZAEB7KLLGCDGBPOF22NSAWH', 'name': 'projects/ee-dhanikondav/operations/VZAEB7KLLGCDGBPOF22NSAWH'}

Started export of high carbon sink areas
Task ID: FMFHW24WRCGRGXLQUL5WHXWO
Status: {'state': 'READY', 'description': 'chilika_high_carbon_sink', 'priority': 100, 'creation_timestamp_ms': 1741751692934, 'update_timestamp_ms': 1741751692934, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'FMFHW24WRCGRGXLQUL5WHXWO', 'name': 'projects/ee-dhanikondav/operations/FMFHW24WRCGRGXLQUL5WHXWO'}

Started export of carbon statistics
Task ID: RY4KEG5XPD4SWFKWXWYV34FU
Status: {'state': 'READY', 'description': 'chilika_carbon_stats', 'priority'